In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as mpl

# importing file for analysis
json_file = './data/sensor1_sleep.json'

with open(json_file, 'r') as infile:
  json_data = json.load(infile)

In [ ]:
type(json_data)

In [ ]:
for element in json_data:
  print(element)

In [ ]:
type(json_data['sleep'])

In [ ]:
first_element = json_data['sleep'][0]
first_element

In [ ]:
# find the element related to the day of interest

days = len(json_data['sleep'])
days

In [ ]:
for i in range(days):
    print(json_data['sleep'][i]['bedtime_end'])

## find time the participant went to bed, and when they woke up 


In [ ]:
night_index = 2
print('Night starts at: '+json_data['sleep'][night_index]['bedtime_start'])
print('Night end at: '+json_data['sleep'][night_index]['bedtime_end'])

In [ ]:
# total time in bed

start = json_data['sleep'][night_index]['bedtime_start']
end = json_data['sleep'][night_index]['bedtime_end']

start_hour = int(start.split('T')[1][:2])
end_hour = int(end.split('T')[1][:2])

total_time = 24-start_hour + end_hour
total_time

In [ ]:
# extract the hypnogram !

#'1' = deep (N3) sleep - '2' = light (N1 or N2) sleep - '3' = REM sleep - '4' = awake
dic_sleep = {'wake':4 , 'deep':1, 'light':2 , 'rem':3}

hypno_js = json_data['sleep'][night_index]['hypnogram_5min']

hypno = np.array(list(hypno_js)) 

# we have 1 value every 5 minutes; we need 1 value every 30 seconds (to compare it with gold standard)
# let's have each element repeated 10 times (2 per minute)
hypno = np.repeat(hypno,10)
hypno = hypno.astype(int)

# hypno is multiple lists that show the phase every 30 seconds
hypno

In [ ]:
# verify hypnogram is the right length 
len(hypno)

In [ ]:
# write down the hypnogram to export,
total_number_30sec_int = 7*60*2

df = pd.DataFrame(columns = ['IndexTime','SleepPhase'])
df['IndexTime'] = range(total_number_30sec_int)

# cut the first and last minutes of hypno (outside the range of interest)
minutes_before_11pm = 6
minutes_after_6am = 9

df['SleepPhase'] = hypno[minutes_before_11pm*2:-minutes_after_6am*2]

In [ ]:
# save the final csv file
df.to_csv('sensor_1_output.csv', index=False)

In [ ]:
# calculate amount of sleep per phase
# in number of intervals
# in percentage over the 7 hours of analysis

deep = df[df.get('SleepPhase') == 1].shape[0]/df.shape[0]
light = df[df.get('SleepPhase') == 2].shape[0]/df.shape[0]
rem = df[df.get('SleepPhase') == 3].shape[0]/df.shape[0]
awake = df[df.get('SleepPhase') == 4].shape[0]/df.shape[0]

print('deep: ' + str(deep))
print('light: ' + str(light))
print('rem: ' + str(rem))
print('awake: ' + str(awake))

In [ ]:
# print hypnogram

def plot_hypno(signal): #,color
  x = np.arange(len(signal))
  y = signal
  
  fontsizeV = 12
  mpl.plot(x,y) #color=color
  mpl.xlabel('Time (30sec intervals)',fontsize=fontsizeV)
  mpl.ylabel('Sleep phase',fontsize=fontsizeV)
  mpl.yticks([1,2,3,4])
  mpl.title('1=deep(N3), 2=light(N1=N2), 3=REM, 4=awake')
  mpl.grid(True)
  mpl.tight_layout()
  
plot_hypno(hypno)